Lesson 11 - Distributed and parallel computing with Dask
========================================================

<p>
<font size=+1>
[https://anaconda.org/datasciencepythonr/11-distributed-and-parallel-computing-with-dask
](https://anaconda.org/datasciencepythonr/11-distributed-and-parallel-computing-with-dask)</font>
</p>

11.1 Describe Dask in relation to Pandas
----------------------------------------


<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="left"
     valign="center"
     width="20%"
     alt="Dask logo">
     
<img src="http://www.numfocus.org/uploads/6/0/6/9/60696727/6893890_orig.png"
     align="right"
     valign="bottom"
     width="40%"
     alt="Pandas logo">
     

Required: [New York City Yellow Cab trip data](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml) -- pick one month (about 1.6 GB per CSV file)

Install `dask` and `distributed` Conda packages using Anaconda Navigator or from the command line (make sure they go into the Conda environment you are using for this course):

    conda install dask distributed
    
Make sure you have that New York City taxi data available and that you know the path.  Correct the paths below to wherever you have placed the data file.  Remember, it is pretty big!

In [ ]:
!ls -skh '../data/nyctaxi/yellow_tripdata_2016-01.csv'

In [ ]:
import pandas as pd

pd.options.display.max_rows = 10

In [ ]:
taxi_fp = '../data/nyctaxi/yellow_tripdata_2016-01.csv'

In [ ]:
with open(taxi_fp) as f:
    df = pd.read_csv(f, nrows=5)
    
df

In [ ]:
%%time
with open(taxi_fp) as f:
    pandas_df = pd.read_csv(f)

In [ ]:
pandas_df

In [ ]:
pandas_df.info()

In [ ]:
import sys
sys.getsizeof(pandas_df) / 2**30 # GB

In [ ]:
del pandas_df

11.2 Profile the creation of Dask dataframes
--------------------------------------------

In [ ]:
import dask.dataframe as dd

from dask.distributed import progress
from distributed import Client

In [ ]:
client = Client()

*Dask Profiler* will now be available at [`http://localhost:8787/status/`](http://localhost:8787/status/)

In [ ]:
%%time
df = dd.read_csv(taxi_fp, 
                 parse_dates=['tpep_pickup_datetime', 
                              'tpep_dropoff_datetime'])

In [ ]:
df = client.persist(df)

After you execute the cell below be sure to check the *Dask Profiler* at [`http://localhost:8787/status/`](http://localhost:8787/status/) where you can also watch the detailed progress of the execution.

In [ ]:
progress(df)

### Dask.dataframe looks *almost* identical to Pandas


In [ ]:
df

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
%%time
df.passenger_count.sum().compute()

In [ ]:
%%time
# Compute average trip distance grouped by passenger count
df.groupby(df.passenger_count).trip_distance.mean().compute()

11.3 Analyze and plot Dask data
-------------------------------

In [ ]:
%%time
# clean the data
df2 = df[(df.tip_amount > 0) & (df.fare_amount > 0)]

In [ ]:
%%time
# add a dimension
df2 = df2.assign(tip_fraction=df2.tip_amount / df2.fare_amount)

In [ ]:
%%time
dayofweek = df2.groupby(df2.tpep_pickup_datetime.dt.dayofweek).tip_fraction.mean() 
hour      = df2.groupby(df2.tpep_pickup_datetime.dt.hour).tip_fraction.mean()

In [ ]:
dayofweek, hour = client.persist([dayofweek, hour])

progress(dayofweek, hour)

### Plot results

In [ ]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

In [ ]:
fig = figure(title='Tip Fraction', 
             x_axis_label='Hour of day', 
             y_axis_label='Tip Fraction',
             width=600,
             height=300)
fig.line(x=hour.index.compute(), y=hour.compute(), line_width=3)
fig.y_range.start = 0

show(fig)

### Dask and Scikit-Learn
You can read about how Dask can be used to parallelize Scikit-Learn model training and prediction.

* [Model parallelism](http://jcrist.github.io/dask-sklearn-part-1.html)
* [Data parallelism](http://jcrist.github.io/dask-sklearn-part-2.html)
* [Dask-Learn parallel gradient descent model](http://jcrist.github.io/dask-sklearn-part-3.html)